In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import h5py
subhalo = h5py.File('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127/fof_subhalo_tab_127.0.hdf5','r')

def printname(name):
    print (name)
    
subhalo.visit(printname)

Config
Group
Group/GroupBHMass
Group/GroupBHMdot
Group/GroupCM
Group/GroupFirstSub
Group/GroupGasMetalFractions
Group/GroupGasMetallicity
Group/GroupLen
Group/GroupLenType
Group/GroupMass
Group/GroupMassType
Group/GroupNsubs
Group/GroupPos
Group/GroupRadioLuminosity
Group/GroupSFR
Group/GroupStarMetalFractions
Group/GroupStarMetallicity
Group/GroupVel
Group/GroupWindMass
Group/GroupXrayLuminosity
Group/Group_M_Crit200
Group/Group_M_Crit500
Group/Group_M_Mean200
Group/Group_M_TopHat200
Group/Group_R_Crit200
Group/Group_R_Crit500
Group/Group_R_Mean200
Group/Group_R_TopHat200
Header
IDs
Parameters
Subhalo
Subhalo/SubhaloBHMass
Subhalo/SubhaloBHMdot
Subhalo/SubhaloBfldDisk
Subhalo/SubhaloBfldHalo
Subhalo/SubhaloCM
Subhalo/SubhaloGasMetalFractions
Subhalo/SubhaloGasMetalFractionsHalfRad
Subhalo/SubhaloGasMetalFractionsMaxRad
Subhalo/SubhaloGasMetalFractionsSfr
Subhalo/SubhaloGasMetalFractionsSfrWeighted
Subhalo/SubhaloGasMetallicity
Subhalo/SubhaloGasMetallicityHalfRad
Subhalo/SubhaloGasMet

In [29]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = pathlib.Path('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/snapdir_127')


def load(path):
    with h5py.File(path,'r') as snap:
        pstr = snap['PartType4/Coordinates'][()]
        pdrk = snap['PartType1/Coordinates'][()]
        pgas = snap['PartType0/Coordinates'][()]
        mstr = snap['PartType4/Masses'][()]
        mdrk = snap['PartType1/Masses'][()]
        mgas = snap['PartType0/Masses'][()]
        
        a = snap.get('Header').attrs['Time']
        h = snap.get('Header').attrs['HubbleParam']


    return {'StrPos' : pstr, 
            'DrkPos' : pdrk, 
            'GasPos' : pgas, 
            'StrMass': mstr,
            'DrkMass': mdrk,
            'GasMass': mgas,
            'aexp'   : a,
            'h'      : h}

with joblib.Parallel(n_jobs=8) as P:
    results = P(
        joblib.delayed(load)(path)
        for path in dir_path.glob('*.hdf5'))
    
# results = np.vstack(results)

gc.collect()

CPU times: user 11.7 s, sys: 45.1 s, total: 56.7 s
Wall time: 1min 8s


150

In [30]:
aexp = results[0]['aexp']
h    = results[0]['h']

pos_str  = np.concatenate([r['StrPos']for r in results])
pos_drk  = np.concatenate([r['DrkPos']for r in results])
pos_gas  = np.concatenate([r['GasPos']for r in results])

pos_str = pos_str*aexp*1000/h
pos_drk = pos_drk*aexp*1000/h
pos_gas = pos_gas*aexp*1000/h

mass_str = np.concatenate([r['StrMass']for r in results])
mass_drk = np.concatenate([r['DrkMass']for r in results])
mass_gas = np.concatenate([r['GasMass']for r in results])

mstr = mass_str*1e10/h
mdrk = mass_drk*1e10/h
mgas = mass_gas*1e10/h


In [31]:
%%time
import gc
import joblib
import h5py
import pathlib
import numpy as np

dir_path = pathlib.Path('/store/clues/HESTIA/RE_SIMS/8192/GAL_FOR/17_11/output_2x2.5Mpc/groups_127')


def load(path):
    with h5py.File(path,'r') as snap:
        sub_mass = snap['Subhalo/SubhaloMass'][()]
        sub_pos  = snap['Subhalo/SubhaloPos'][()]
        
        Omega0   = snap.get('Header').attrs['Omega0']
        OmegaL   = snap.get('Header').attrs['OmegaLambda']
        Redshift = snap.get('Header').attrs['Redshift']
    
    return {'SubMass': sub_mass, 
            'SubPos' : sub_pos,
            
            'Omega0'  : Omega0,
            'OmegaL'  : OmegaL,
            'Redshift': Redshift}

with joblib.Parallel(n_jobs=8) as P:
    results = P(
        joblib.delayed(load)(path)
        for path in dir_path.glob('*.hdf5'))
    
# results = np.vstack(results)
gc.collect()

CPU times: user 77.3 ms, sys: 1.79 s, total: 1.87 s
Wall time: 1.87 s


142

In [32]:
Smass = np.concatenate([r['SubMass']for r in results])
Spos  = np.concatenate([r['SubPos']for r in results])

sort = np.argsort(Smass)

SMASS = Smass[sort][::-1]*1e10/h
SPOS  = Spos*aexp*1000/h

In [ ]:
%%time

import numpy as np
import gc

G = 4.299e-6 #kpc*(km/s)*Msol^-1

H0 = 100*h #km/s/Mpc

omega_matter = results[0]['Omega0']
omega_lambda = results[0]['OmegaL']
omega0 = omega_matter + omega_lambda

z = results[0]['Redshift']

nsub = 100
Mvir = np.zeros(nsub)
Rvir = np.zeros(nsub)

for i in range(nsub):
    
    xstr = pos_str[:,0] - SPOS[i,0]
    ystr = pos_str[:,1] - SPOS[i,1]
    zstr = pos_str[:,2] - SPOS[i,2]
    rstr = np.sqrt(xstr**2 + ystr**2 + zstr**2)

    xdrk = pos_drk[:,0] - SPOS[i,0]
    ydrk = pos_drk[:,1] - SPOS[i,1]
    zdrk = pos_drk[:,2] - SPOS[i,2]
    rdrk = np.sqrt(xdrk**2 + ydrk**2 + zdrk**2)
    
    xgas = pos_gas[:,0] - SPOS[i,0]
    ygas = pos_gas[:,1] - SPOS[i,1]
    zgas = pos_gas[:,2] - SPOS[i,2]
    rgas = np.sqrt(xgas**2 + ygas**2 + zgas**2)
    
    st, = np.where(rstr < 250)
    dm, = np.where(rdrk < 250)
    gs, = np.where(rstr < 250)
    
    r = np.concatenate([rstr[st], rgas[gs], rdrk[dm]])
    m = np.concatenate([mstr[st], mgas[gs], mdrk[dm]])

    psort = np.argsort(r)

    vol  = 4./3 * np.pi * r[psort]**3
    mass = np.cumsum(m[psort])
    rho  = mass / vol

    H0_kpc = H0/1000 #paso a kpc
    
    Ht = H0_kpc*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
    
    rho_crit = 3*Ht**2 / (8*np.pi*G)

    mask, = np.where(rho < 200*rho_crit)
    
    Mvir[i] = mass[mask][0]
    Rvir[i] = r[psort][mask][0] 
    
    print ('subhalo', str('%03d'%i), 'Mvir = ', str('%12.8f'%Mvir[i]), 'Rvir = ', str('%12.8f'%Rvir[i]))
    
    gc.collect()
    

subhalo 000 Mvir =  1868064227328.00000000 Rvir =  259.55519224
subhalo 001 Mvir =  183293755392.00000000 Rvir =  119.71459669
subhalo 002 Mvir =  1567603294208.00000000 Rvir =  244.81774658
subhalo 003 Mvir =  118834708480.00000000 Rvir =  103.61249580
subhalo 004 Mvir =  28163397632.00000000 Rvir =   64.11993422
subhalo 005 Mvir =  153759072256.00000000 Rvir =  112.90448575
subhalo 006 Mvir =  1830560858112.00000000 Rvir =  257.80813759
subhalo 007 Mvir =  1752400265216.00000000 Rvir =  254.08320695
subhalo 008 Mvir =  3358010368.00000000 Rvir =   31.56067114
subhalo 009 Mvir =  18452619264.00000000 Rvir =   55.69103455
subhalo 010 Mvir =  3871873024.00000000 Rvir =   33.09630826


In [17]:
(1.88e-29)*h**2 *(3.086e21)**3 / (1.989e33) 

127.58094145367797

In [39]:
print (rho_crit*200)

25504.62350081994


In [41]:
print (r[psort][mask])

[  259.55519224   259.58658917   259.59027663 ... 90929.78325109
 91228.94830954 91241.25829807]


In [ ]:
for i in range(100):
    print (SMASS[i], Mvir[i])

In [ ]:

r = np.concatenate(rstr,rgas,rdrk)
m = np.concatenate(mstr,mgas,mdrk)

psort = np.argsort(r)

vol  = 4./3 * np.pi * r[psort]**3
mass = m[psort]
rho  = mass / vol

rho_crit = 3*H**2 / (8*np.pi*G)

mask, = np.where(rho < 200*rho_crit)
Mvir = mvir[-1]

In [ ]:
h = 0.6777
G = 4.299e-6 #kpc*(km/s)*Msol^-1
a0 = 1
H0 = h*100
omega_lambda = 0.682
omega_matter = 0.318
omega0 = omega_matter + omega_lambda

z = a0/aexp[isnap] - 1.
Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)
time = tiempo.conv(z, h, omega_lambda, omega_matter)

#---aca paso las coordenadas respecto al centro de la galaxia------
xstr = (pstr[:,0]-cm[0])*aexp[isnap]/h
ystr = (pstr[:,1]-cm[1])*aexp[isnap]/h
zstr = (pstr[:,2]-cm[2])*aexp[isnap]/h

v_x = vel[:,0] *np.sqrt(aexp[isnap]) + Ht *xstr/1000.
v_y = vel[:,1] *np.sqrt(aexp[isnap]) + Ht *ystr/1000.
v_z = vel[:,2] *np.sqrt(aexp[isnap]) + Ht *zstr/1000.